In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import log_loss, roc_auc_score
from deepctr.models import DeepFM,PNN
from deepctr.models import FGCNN
from deepctr.inputs import  SparseFeat, DenseFeat,get_feature_names
import math
import numpy as np
from keras.callbacks import EarlyStopping
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [11]:
import numpy as np
data = pd.read_csv('./dac/avazu/avazu_sampled.csv')

data=data.astype({'hour': 'int32'})
dense_features = ["C1"]+['C'+str(i) for i in range(14, 22)]
sparse_features = ["C1"]+['C'+str(i) for i in range(14, 22)]  +["hour","banner_pos","site_id","site_domain","site_category","app_id","app_domain","app_category","device_id","device_ip","device_model","device_type","device_conn_type"]

# 
data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0,)
target = 'click'

  
for feat in dense_features:
    data[feat]=pd.cut(data[feat], 40)
    
# 1.Label Encoding for sparse features,and do simple Transformation for dense features
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat]=data[feat].apply(lambda x: str(x))
    data[feat] = lbe.fit_transform(data[feat])
    

    
# mms = MinMaxScaler(feature_range=(0, 1))
# data[dense_features] = mms.fit_transform(data[dense_features])

# 2.count #unique features for each sparse field,and record dense feature field name

fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique(),embedding_dim=40)
                       for i,feat in enumerate(sparse_features)] 
# + [DenseFeat(feat, 1,) for feat in dense_features]
print(np.array(fixlen_feature_columns).shape)
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 3.generate input data for model

train, test = train_test_split(data, test_size=0.2)

train_model_input = {name:train[name] for name in feature_names}
test_model_input = {name:test[name] for name in feature_names}

(22, 7)


In [4]:
import kerastuner
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow import keras

def buildxDeepFM(hp):
#     activation = hp.Choice('activation', ['relu', 'selu','elu'])
#     dropout_rate = hp.Float(
#                 'dropout_rate', 0.0, 0.6, step=0.1, default=0.5)
    model = xDeepFM(linear_feature_columns, dnn_feature_columns,dnn_dropout=0.8, task='binary')
    model.compile( optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), loss="binary_crossentropy",metrics=['binary_crossentropy'])
    return model
def buildDeepFM(hp):
    activation = hp.Choice('activation', ['relu', 'selu','elu'])
    dropout_rate = hp.Float(
                'dropout_rate', 0.0, 0.6, step=0.1, default=0.5)
    model = DeepFM(linear_feature_columns, dnn_feature_columns,dnn_dropout=0.8, task='binary')
    model.compile( optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), loss="binary_crossentropy",metrics=['binary_crossentropy'])
    return model

def buildPNN(hp):
    activation = hp.Choice('activation', ['relu', 'selu','elu'])
    dropout_rate = hp.Float(
                'dropout_rate', 0.0, 0.6, step=0.1, default=0.5)
    model = PNN(linear_feature_columns, dnn_feature_columns,dnn_dropout=0.8, task='binary')
    model.compile( optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), loss="binary_crossentropy",metrics=['binary_crossentropy'])
    return model
    
def buildFNN(hp):
    activation = hp.Choice('activation', ['relu', 'selu','elu'])
    dropout_rate = hp.Float(
                'dropout_rate', 0.0, 0.6, step=0.1, default=0.5)
    model = FNN(linear_feature_columns, dnn_feature_columns,dnn_dropout=0.8, task='binary')
    model.compile( optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4,1e-4])), loss="binary_crossentropy",metrics=['binary_crossentropy'])
    return model

def buildFGCNN(hp):
#     k=40
#     conv=7*1
#     Kernal=[14,16,18,20]
#     new=[3,3,3,3]
#     ruenet=[4096,2048,1/]
    activation = hp.Choice('activation', ['relu', 'selu','elu'])
    dropout_rate = hp.Float(
                'dropout_rate', 0.0, 0.8, step=0.1, default=0.5)
    model = FGCNN(linear_feature_columns, dnn_feature_columns,dnn_dropout=0.8, task='binary')
    model.compile( optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4,1e-4])), loss="binary_crossentropy",metrics=['binary_crossentropy'])
    return model

def buildAutoint(hp):
   
    activation = hp.Choice('activation', ['relu', 'selu','elu'])
    dropout_rate = hp.Float(
                'dropout_rate', 0.0, 0.8, step=0.1, default=0.5)
    model=AutoInt(linear_feature_columns, dnn_feature_columns, dnn_use_bn=True, dnn_dropout=0, task='binary' )
    model.compile( optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4,1e-4])), loss="binary_crossentropy",metrics=['binary_crossentropy'])
    return model
    
    
    
class MyTuner(kerastuner.tuners.BayesianOptimization):
    def run_trial(self, trial, *args, **kwargs):
        # You can add additional HyperParameters for preprocessing and custom training loops
        # via overriding `run_trial`
        kwargs['batch_size'] = trial.hyperparameters.Int('batch_size', 256, 1024, step=256)
        kwargs['epochs'] = trial.hyperparameters.Int('epochs', 10, 30, step=5)
        super(MyTuner, self).run_trial(trial, *args, **kwargs)
        
def HypermeterTune(model,directory):
    tuner = MyTuner(
        model,
        objective='val_binary_crossentropy',
        max_trials=2,
        executions_per_trial=2,
        directory=directory)

    tuner.search_space_summary()

    tuner.search(x=train_model_input,
                 y=train[target].values,callbacks=[tf.keras.callbacks.EarlyStopping('val_binary_crossentropy', patience=1)],
                 validation_data=(test_model_input,test[target].values))

    return tuner


In [5]:
import itertools

import tensorflow as tf
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.initializers import (Zeros, glorot_normal,
                                                  glorot_uniform)
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras.regularizers import l2
from tensorflow.python.layers import utils

from deepctr.layers.activation import activation_layer
from deepctr.layers.utils import concat_func,reduce_sum,softmax,reduce_mean


class FGCNNLayer(Layer):
    """Feature Generation Layer used in FGCNN,including Convolution,MaxPooling and Recombination.

      Input shape
        - A 3D tensor with shape:``(batch_size,field_size,embedding_size)``.

      Output shape
        - 3D tensor with shape: ``(batch_size,new_feture_num,embedding_size)``.

      References
        - [Liu B, Tang R, Chen Y, et al. Feature Generation by Convolutional Neural Network for Click-Through Rate Prediction[J]. arXiv preprint arXiv:1904.04447, 2019.](https://arxiv.org/pdf/1904.04447)

    """

    def __init__(self, filters=(14, 16,), kernel_width=(7, 7,), new_maps=(3, 3,), pooling_width=(2, 2),
                 **kwargs):
        if not (len(filters) == len(kernel_width) == len(new_maps) == len(pooling_width)):
            raise ValueError("length of argument must be equal")
        self.filters = filters
        self.kernel_width = kernel_width
        self.new_maps = new_maps
        self.pooling_width = pooling_width

        super(FGCNNLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        print("FGCNN Layer start")
        if len(input_shape) != 3:
            raise ValueError(
                "Unexpected inputs dimensions %d, expect to be 3 dimensions" % (len(input_shape)))
        self.conv_layers = []
        self.pooling_layers = []
        self.dense_layers = []
        pooling_shape = input_shape.as_list() + [1, ]
        embedding_size = int(input_shape[-1])
        for i in range(1, len(self.filters) + 1):
            print(i)
            filters = self.filters[i - 1]
            width = self.kernel_width[i - 1]
            new_filters = self.new_maps[i - 1]
            pooling_width = self.pooling_width[i - 1]
            conv_output_shape = self._conv_output_shape(
                pooling_shape, (width, 1))
            pooling_shape = self._pooling_output_shape(
                conv_output_shape, (pooling_width, 1))
            
            self.conv_layers.append(tf.keras.layers.Conv2D(filters=filters, kernel_size=(width, 1), strides=(1, 1),
                                                           padding='same',
                                                           activation='tanh', use_bias=True, ))
            self.pooling_layers.append(
                tf.keras.layers.MaxPooling2D(pool_size=(pooling_width, 1)))
            self.dense_layers.append(tf.keras.layers.Dense(pooling_shape[1] * embedding_size * new_filters,
                                                           activation='tanh', use_bias=True))

        self.flatten = tf.keras.layers.Flatten()

        super(FGCNNLayer, self).build(
            input_shape)  # Be sure to call this somewhere!

    def call(self, inputs, **kwargs):

        if K.ndim(inputs) != 3:
            raise ValueError(
                "Unexpected inputs dimensions %d, expect to be 3 dimensions" % (K.ndim(inputs)))

        embedding_size = int(inputs.shape[-1])
        pooling_result = tf.expand_dims(inputs, axis=3)

        new_feature_list = []

        for i in range(1, len(self.filters) + 1):
            new_filters = self.new_maps[i - 1]

            conv_result = self.conv_layers[i - 1](pooling_result)

            pooling_result = self.pooling_layers[i - 1](conv_result)

            flatten_result = self.flatten(pooling_result)

            new_result = self.dense_layers[i - 1](flatten_result)

            new_feature_list.append(tf.reshape(new_result, (-1, int(pooling_result.shape[1]) * new_filters, embedding_size)))
            print(new_result.shape)
        new_features = concat_func(new_feature_list, axis=1)
        return new_features

    def compute_output_shape(self, input_shape):
        new_features_num = 0
        features_num = input_shape[1]

        for i in range(0, lren(self.kernel_width)):
            pooled_features_numr = features_num // self.pooling_width[i]
            new_features_num += self.new_maps[i] * pooled_features_num
            features_num = pooled_features_num

        return (None, new_features_num, input_shape[-1])

    def get_config(self, ):
        config = {'kernel_width': self.kernel_width, 'filters': self.filters, 'new_maps': self.new_maps,
                  'pooling_width': self.pooling_width}
        base_config = super(FGCNNLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def _conv_output_shape(self, input_shape, kernel_size):
        # channels_last
        space = input_shape[1:-1]
        new_space = []
        for i in range(len(space)):
            new_dim = utils.conv_output_length(
                space[i],
                kernel_size[i],
                padding='same',
                stride=1,
                dilation=1)
            new_space.append(new_dim)
        return ([input_shape[0]] + new_space + [self.filters])

    def _pooling_output_shape(self, input_shape, pool_size):
        # channels_last
        rows = input_shape[1]
        cols = input_shape[2]
        rows = utils.conv_output_length(rows, pool_size[0], 'valid',
                                        pool_size[0])
        cols = utils.conv_output_length(cols, pool_size[1], 'valid',
                                        pool_size[1])
        return [input_shape[0], rows, cols, input_shape[3]]

In [6]:
import tensorflow as tf


from deepctr.inputs import build_input_features, input_from_feature_columns, get_linear_logit
from deepctr.layers.core import PredictionLayer, DNN
from deepctr.layers.interaction import InnerProductLayer, FGCNNLayer
from deepctr.layers.utils import concat_func, add_func


def unstack(input_tensor):
    input_ = tf.expand_dims(input_tensor, axis=2)
    return tf.unstack(input_, input_.shape[1], 1)


def FGCNN(linear_feature_columns,dnn_feature_columns, conv_kernel_width=(7, 7, 7, 7), conv_filters=(14, 16, 18,20),
          new_maps=(3, 3, 3,3),
          pooling_width=(2, 2, 2,2), dnn_hidden_units=(128,),l2_reg_linear=1e-5, l2_reg_embedding=1e-5, l2_reg_dnn=0, dnn_dropout=0,
          init_std=0.0001, seed=1024,
          task='binary', ):
    """Instantiates the Feature Generation by Convolutional Neural Network architecture.
    :param linear_feature_columns: An iterable containing all the features used by linear part of the model.
    :param dnn_feature_columns: An iterable containing all the features used by deep part of the model.
    :param conv_kernel_width: list,list of positive integer or empty list,the width of filter in each conv layer.
    :param conv_filters: list,list of positive integer or empty list,the number of filters in each conv layer.
    :param new_maps: list, list of positive integer or empty list, the feature maps of generated features.
    :param pooling_width: list, list of positive integer or empty list,the width of pooling layer.
    :param dnn_hidden_units: list,list of positive integer or empty list, the layer number and units in each layer of deep net.
    :param l2_reg_linear: float. L2 regularizer strength applied to linear part
    :param l2_reg_embedding: float. L2 regularizer strength applied to embedding vector
    :param l2_reg_dnn: float. L2 regularizer strength applied to DNN
    :param dnn_dropout: float in [0,1), the probability we will drop out a given DNN coordinate.
    :param init_std: float,to use as the initialize std of embedding vector
    :param seed: integer ,to use as random seed.
    :param task: str, ``"binary"`` for  binary logloss or  ``"regression"`` for regression loss
    :return: A Keras model instance.
    """
    print("FGCNN start")
    if not (len(conv_kernel_width) == len(conv_filters) == len(new_maps) == len(pooling_width)):
        raise ValueError(
            "conv_kernel_width,conv_filters,new_maps  and pooling_width must have same length")

    features = build_input_features(dnn_feature_columns)

    inputs_list = list(features.values())

    linear_logit = get_linear_logit(features, linear_feature_columns, init_std=init_std, seed=seed, prefix='linear',
                                    l2_reg=l2_reg_linear)

    deep_emb_list, _ = input_from_feature_columns(features, dnn_feature_columns, l2_reg_embedding, init_std, seed)
    fg_deep_emb_list,_ = input_from_feature_columns(features, dnn_feature_columns, l2_reg_embedding, init_std, seed,
                                                    prefix='fg')


    fg_input = concat_func(fg_deep_emb_list, axis=1)
    origin_input = concat_func(deep_emb_list, axis=1)
    print("origin_input shape:",origin_input.shape)
    if len(conv_filters) > 0:
        new_features = FGCNNLayer(
            conv_filters, conv_kernel_width, new_maps, pooling_width)(fg_input)
        combined_input = concat_func([origin_input, new_features], axis=1)
    else:
        combined_input = origin_input
    inner_product = tf.keras.layers.Flatten()(InnerProductLayer()(
        tf.keras.layers.Lambda(unstack, mask=[None] * int(combined_input.shape[1]))(combined_input)))
    linear_signal = tf.keras.layers.Flatten()(combined_input)
    dnn_input = tf.keras.layers.Concatenate()([linear_signal, inner_product])
    dnn_input = tf.keras.layers.Flatten()(dnn_input)

    final_logit = DNN(dnn_hidden_units, dropout_rate=dnn_dropout,
                      l2_reg=l2_reg_dnn)(dnn_input)
    final_logit = tf.keras.layers.Dense(1, use_bias=False)(final_logit)

    final_logit = add_func([final_logit,linear_logit])
    output = PredictionLayer(task)(final_logit)

    model = tf.keras.models.Model(inputs=inputs_list, outputs=output)
    return model

In [10]:
# 4.Define Model,train,predict and evaluate
model = FGCNN(linear_feature_columns, dnn_feature_columns,conv_kernel_width=(7, 7, 7), conv_filters=(14, 16, 18),
          new_maps=(3, 3, 3),
          pooling_width=( 2, 2,2), task='binary')
model.compile("adam", "binary_crossentropy",metrics=['binary_crossentropy'])
earlystopping_callback = EarlyStopping(monitor='val_binary_crossentropy',verbose=1,patience=1,baseline=None)
history = model.fit(train_model_input, train[target].values,
                    batch_size=1024, epochs=15, verbose=2, validation_split=0.2)
pred_ans = model.predict(test_model_input, batch_size=256)

FGCNN start
origin_input shape: (None, 13, 40)
Train on 334458 samples, validate on 83615 samples
Epoch 1/15


/home/yxy/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/yxy/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


KeyboardInterrupt: 

In [36]:
import kerastuner
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow import keras
def buildFGCNN(hp):
    # k=40
    # conv=7*1
    #Kernal=[14,16,18,20]
    #new=[3,3,3,3]
    # ruenet=[4096,2048,1/]
    activation = hp.Choice('activation', ['relu', 'selu','elu'])
    dropout_rate = hp.Float(
                'dropout_rate', 0.0, 0.8, step=0.1, default=0.5)
    model = FGCNN(linear_feature_columns, dnn_feature_columns,conv_kernel_width=(7, 7, 7), conv_filters=(12, 14, 16),
          new_maps=(3, 3, 3),
          pooling_width=( 2, 2,2),dnn_hidden_units=(4096,12048,1024,512,1,),dnn_dropout=0.4, task='binary')

    model.compile( optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4,1e-4])), loss="binary_crossentropy",metrics=['binary_crossentropy'])
    return model


class MyTuner(kerastuner.tuners.BayesianOptimization):
    def run_trial(self, trial, *args, **kwargs):
        # You can add additional HyperParameters for preprocessing and custom training loops
        # via overriding `run_trial`
        kwargs['batch_size'] = trial.hyperparameters.Int('batch_size', 256, 1024, step=256)
        kwargs['epochs'] = trial.hyperparameters.Int('epochs', 10, 30, step=5)
        super(MyTuner, self).run_trial(trial, *args, **kwargs)
  
def HypermeterTune(model,directory):
    tuner = MyTuner(
        buildFGCNN,
        objective='val_binary_crossentropy',
        max_trials=5,
        executions_per_trial=2,
        directory=directory)

    tuner.search_space_summary()

    tuner.search(x=train_model_input,
                 y=train[target].values,callbacks=[tf.keras.callbacks.EarlyStopping('val_binary_crossentropy', patience=1)],
                 validation_data=(test_model_input,test[target].values))

    return tuner


In [37]:
def bestModel(tuner,batch_size,modelName):
    hps = tuner.get_best_hyperparameters()[0]
    es = EarlyStopping('val_binary_crossentropy', patience=1)
    best_model = tuner.hypermodel.build(hps)
    #callbacks=[es],
    history = best_model.fit(train_model_input, train[target].values,
                                batch_size=batch_size, epochs=20, verbose=2,validation_split=0.2)
    pred_ans = best_model.predict(test_model_input, batch_size=256)
    print("Final Result of ",modelName, " :")
    print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
    print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

In [40]:
FGCNNtuner=HypermeterTune("buildFGCNN","AvazuFGCNN")

INFO:tensorflow:Reloading Oracle from AvazuFGCNN/untitled_project/oracle.json
FGCNN start
origin_input shape: (None, 13, 40)
INFO:tensorflow:Reloading Tuner from AvazuFGCNN/untitled_project/tuner0.json


FGCNN start
origin_input shape: (None, 13, 40)
Train on 418073 samples, validate on 104519 samples
Epoch 1/15


/home/yxy/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/yxy/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


112128/418073 [=======>......................] - ETA: 47:19 - loss: 1.0470 - binary_crossentropy: 1.04 - ETA: 26:37 - loss: 0.9350 - binary_crossentropy: 0.93 - ETA: 19:38 - loss: 0.8127 - binary_crossentropy: 0.81 - ETA: 16:07 - loss: 0.8794 - binary_crossentropy: 0.87 - ETA: 14:03 - loss: 0.8093 - binary_crossentropy: 0.80 - ETA: 12:42 - loss: 0.7541 - binary_crossentropy: 0.75 - ETA: 11:40 - loss: 0.7135 - binary_crossentropy: 0.71 - ETA: 10:53 - loss: 0.6819 - binary_crossentropy: 0.68 - ETA: 10:17 - loss: 0.6584 - binary_crossentropy: 0.65 - ETA: 9:50 - loss: 0.6380 - binary_crossentropy: 0.6380 - ETA: 9:26 - loss: 0.6192 - binary_crossentropy: 0.619 - ETA: 9:07 - loss: 0.6049 - binary_crossentropy: 0.604 - ETA: 8:50 - loss: 0.5962 - binary_crossentropy: 0.596 - ETA: 8:35 - loss: 0.5851 - binary_crossentropy: 0.585 - ETA: 8:22 - loss: 0.5749 - binary_crossentropy: 0.574 - ETA: 8:12 - loss: 0.5691 - binary_crossentropy: 0.569 - ETA: 8:02 - loss: 0.5604 - binary_crossentropy: 0.560 

224256/418073 [===============>..............] - ETA: 4:22 - loss: 0.4198 - binary_crossentropy: 0.418 - ETA: 4:22 - loss: 0.4198 - binary_crossentropy: 0.418 - ETA: 4:21 - loss: 0.4196 - binary_crossentropy: 0.418 - ETA: 4:20 - loss: 0.4192 - binary_crossentropy: 0.418 - ETA: 4:19 - loss: 0.4191 - binary_crossentropy: 0.418 - ETA: 4:18 - loss: 0.4187 - binary_crossentropy: 0.417 - ETA: 4:18 - loss: 0.4185 - binary_crossentropy: 0.417 - ETA: 4:17 - loss: 0.4182 - binary_crossentropy: 0.417 - ETA: 4:16 - loss: 0.4178 - binary_crossentropy: 0.416 - ETA: 4:15 - loss: 0.4178 - binary_crossentropy: 0.416 - ETA: 4:15 - loss: 0.4176 - binary_crossentropy: 0.416 - ETA: 4:14 - loss: 0.4173 - binary_crossentropy: 0.416 - ETA: 4:13 - loss: 0.4170 - binary_crossentropy: 0.416 - ETA: 4:12 - loss: 0.4167 - binary_crossentropy: 0.415 - ETA: 4:12 - loss: 0.4163 - binary_crossentropy: 0.415 - ETA: 4:11 - loss: 0.4163 - binary_crossentropy: 0.415 - ETA: 4:10 - loss: 0.4162 - binary_crossentropy: 0.415 -

...

112128/418073 [=======>......................] - ETA: 6:08 - loss: 0.2802 - binary_crossentropy: 0.238 - ETA: 6:00 - loss: 0.2942 - binary_crossentropy: 0.252 - ETA: 5:53 - loss: 0.2910 - binary_crossentropy: 0.249 - ETA: 5:50 - loss: 0.2932 - binary_crossentropy: 0.251 - ETA: 5:50 - loss: 0.2925 - binary_crossentropy: 0.250 - ETA: 5:51 - loss: 0.2918 - binary_crossentropy: 0.249 - ETA: 5:49 - loss: 0.2909 - binary_crossentropy: 0.248 - ETA: 5:47 - loss: 0.2920 - binary_crossentropy: 0.250 - ETA: 5:47 - loss: 0.2917 - binary_crossentropy: 0.249 - ETA: 5:46 - loss: 0.2939 - binary_crossentropy: 0.251 - ETA: 5:45 - loss: 0.2948 - binary_crossentropy: 0.252 - ETA: 5:45 - loss: 0.2937 - binary_crossentropy: 0.251 - ETA: 5:44 - loss: 0.2946 - binary_crossentropy: 0.252 - ETA: 5:43 - loss: 0.2943 - binary_crossentropy: 0.252 - ETA: 5:44 - loss: 0.2943 - binary_crossentropy: 0.252 - ETA: 5:44 - loss: 0.2946 - binary_crossentropy: 0.252 - ETA: 5:43 - loss: 0.2950 - binary_crossentropy: 0.253 -

224256/418073 [===============>..............] - ETA: 4:25 - loss: 0.3075 - binary_crossentropy: 0.265 - ETA: 4:25 - loss: 0.3074 - binary_crossentropy: 0.265 - ETA: 4:24 - loss: 0.3073 - binary_crossentropy: 0.265 - ETA: 4:23 - loss: 0.3074 - binary_crossentropy: 0.265 - ETA: 4:22 - loss: 0.3073 - binary_crossentropy: 0.265 - ETA: 4:22 - loss: 0.3076 - binary_crossentropy: 0.265 - ETA: 4:21 - loss: 0.3074 - binary_crossentropy: 0.265 - ETA: 4:20 - loss: 0.3075 - binary_crossentropy: 0.265 - ETA: 4:20 - loss: 0.3075 - binary_crossentropy: 0.265 - ETA: 4:19 - loss: 0.3075 - binary_crossentropy: 0.265 - ETA: 4:18 - loss: 0.3075 - binary_crossentropy: 0.265 - ETA: 4:18 - loss: 0.3076 - binary_crossentropy: 0.265 - ETA: 4:17 - loss: 0.3076 - binary_crossentropy: 0.265 - ETA: 4:16 - loss: 0.3077 - binary_crossentropy: 0.265 - ETA: 4:16 - loss: 0.3077 - binary_crossentropy: 0.265 - ETA: 4:15 - loss: 0.3078 - binary_crossentropy: 0.265 - ETA: 4:14 - loss: 0.3079 - binary_crossentropy: 0.265 -

336384/418073 [=======================>......] - ETA: 2:55 - loss: 0.3156 - binary_crossentropy: 0.273 - ETA: 2:54 - loss: 0.3157 - binary_crossentropy: 0.273 - ETA: 2:54 - loss: 0.3157 - binary_crossentropy: 0.273 - ETA: 2:53 - loss: 0.3156 - binary_crossentropy: 0.273 - ETA: 2:52 - loss: 0.3156 - binary_crossentropy: 0.273 - ETA: 2:52 - loss: 0.3158 - binary_crossentropy: 0.273 - ETA: 2:51 - loss: 0.3159 - binary_crossentropy: 0.273 - ETA: 2:50 - loss: 0.3160 - binary_crossentropy: 0.273 - ETA: 2:50 - loss: 0.3160 - binary_crossentropy: 0.273 - ETA: 2:49 - loss: 0.3161 - binary_crossentropy: 0.274 - ETA: 2:48 - loss: 0.3163 - binary_crossentropy: 0.274 - ETA: 2:48 - loss: 0.3163 - binary_crossentropy: 0.274 - ETA: 2:47 - loss: 0.3164 - binary_crossentropy: 0.274 - ETA: 2:46 - loss: 0.3164 - binary_crossentropy: 0.274 - ETA: 2:46 - loss: 0.3164 - binary_crossentropy: 0.274 - ETA: 2:45 - loss: 0.3164 - binary_crossentropy: 0.274 - ETA: 2:44 - loss: 0.3164 - binary_crossentropy: 0.274 -

418073/418073 [==============================] - ETA: 1:15 - loss: 0.3231 - binary_crossentropy: 0.281 - ETA: 1:14 - loss: 0.3232 - binary_crossentropy: 0.281 - ETA: 1:13 - loss: 0.3232 - binary_crossentropy: 0.281 - ETA: 1:12 - loss: 0.3232 - binary_crossentropy: 0.281 - ETA: 1:12 - loss: 0.3233 - binary_crossentropy: 0.281 - ETA: 1:11 - loss: 0.3233 - binary_crossentropy: 0.281 - ETA: 1:10 - loss: 0.3233 - binary_crossentropy: 0.281 - ETA: 1:10 - loss: 0.3234 - binary_crossentropy: 0.281 - ETA: 1:09 - loss: 0.3234 - binary_crossentropy: 0.281 - ETA: 1:08 - loss: 0.3235 - binary_crossentropy: 0.281 - ETA: 1:08 - loss: 0.3235 - binary_crossentropy: 0.281 - ETA: 1:07 - loss: 0.3236 - binary_crossentropy: 0.281 - ETA: 1:06 - loss: 0.3236 - binary_crossentropy: 0.281 - ETA: 1:05 - loss: 0.3236 - binary_crossentropy: 0.281 - ETA: 1:05 - loss: 0.3237 - binary_crossentropy: 0.281 - ETA: 1:04 - loss: 0.3237 - binary_crossentropy: 0.281 - ETA: 1:03 - loss: 0.3237 - binary_crossentropy: 0.281 -

INFO:tensorflow:Oracle triggered exit


In [42]:
FGCNNtuner.results_summary()
print(FGCNNtuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values)
bestModel(FGCNNtuner,1024,"FGCNNtuner")

{'activation': 'selu', 'dropout_rate': 0.8, 'learning_rate': 0.0001, 'batch_size': 1024, 'epochs': 15}
FGCNN start
origin_input shape: (None, 13, 40)
Train on 334458 samples, validate on 83615 samples
Epoch 1/20


/home/yxy/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/yxy/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


334458/334458 - 288s - loss: 0.4672 - binary_crossentropy: 0.4671 - val_loss: 0.3975 - val_binary_crossentropy: 0.3973
Epoch 2/20
334458/334458 - 278s - loss: 0.3868 - binary_crossentropy: 0.3865 - val_loss: 0.3885 - val_binary_crossentropy: 0.3880
Epoch 3/20
334458/334458 - 276s - loss: 0.3721 - binary_crossentropy: 0.3713 - val_loss: 0.3850 - val_binary_crossentropy: 0.3839
Epoch 4/20
334458/334458 - 260s - loss: 0.3610 - binary_crossentropy: 0.3595 - val_loss: 0.3834 - val_binary_crossentropy: 0.3815
Epoch 5/20
334458/334458 - 260s - loss: 0.3517 - binary_crossentropy: 0.3493 - val_loss: 0.3827 - val_binary_crossentropy: 0.3799
Epoch 6/20
334458/334458 - 258s - loss: 0.3437 - binary_crossentropy: 0.3404 - val_loss: 0.3822 - val_binary_crossentropy: 0.3784
Epoch 7/20
334458/334458 - 261s - loss: 0.3369 - binary_crossentropy: 0.3325 - val_loss: 0.3825 - val_binary_crossentropy: 0.3775
Epoch 8/20
334458/334458 - 258s - loss: 0.3309 - binary_crossentropy: 0.3253 - val_loss: 0.3831 - val

In [12]:
model = FGCNN(linear_feature_columns, dnn_feature_columns,conv_kernel_width=(7, 7, 7), conv_filters=(12, 14, 16),
          new_maps=(3, 3, 3),
          pooling_width=( 2, 2,2),dnn_hidden_units=(4096,2048,1024,512,1,),dnn_dropout=0.4, task='binary')
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss="binary_crossentropy",metrics=['binary_crossentropy'])
earlystopping_callback = EarlyStopping(monitor='val_binary_crossentropy',verbose=1,patience=2,baseline=None)
history = model.fit(train_model_input, train[target].values,
                    batch_size=512, epochs=15,callbacks=[earlystopping_callback] ,verbose=2, validation_split=0.2)
pred_ans = model.predict(test_model_input, batch_size=256)

FGCNN start
origin_input shape: (None, 22, 40)
Train on 334458 samples, validate on 83615 samples
Epoch 1/15


/home/yxy/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/yxy/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


334458/334458 - 337s - loss: 0.3972 - binary_crossentropy: 0.3971 - val_loss: 0.3865 - val_binary_crossentropy: 0.3862
Epoch 2/15
334458/334458 - 324s - loss: 0.3695 - binary_crossentropy: 0.3689 - val_loss: 0.3819 - val_binary_crossentropy: 0.3808
Epoch 3/15
334458/334458 - 325s - loss: 0.3550 - binary_crossentropy: 0.3534 - val_loss: 0.3804 - val_binary_crossentropy: 0.3781
Epoch 4/15
334458/334458 - 326s - loss: 0.3436 - binary_crossentropy: 0.3407 - val_loss: 0.3802 - val_binary_crossentropy: 0.3765
Epoch 5/15
334458/334458 - 326s - loss: 0.3343 - binary_crossentropy: 0.3299 - val_loss: 0.3807 - val_binary_crossentropy: 0.3756
Epoch 6/15
334458/334458 - 325s - loss: 0.3266 - binary_crossentropy: 0.3206 - val_loss: 0.3826 - val_binary_crossentropy: 0.3758
Epoch 7/15
334458/334458 - 322s - loss: 0.3204 - binary_crossentropy: 0.3128 - val_loss: 0.3836 - val_binary_crossentropy: 0.3752
Epoch 8/15
334458/334458 - 321s - loss: 0.3149 - binary_crossentropy: 0.3057 - val_loss: 0.3856 - val

In [13]:
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

test LogLoss 0.3783
test AUC 0.7784


In [15]:
model = FGCNN(linear_feature_columns, dnn_feature_columns,conv_kernel_width=(7, 7, 7), conv_filters=(12, 14, 16),
          new_maps=(3, 3, 3),l2_reg_linear=0.0005, l2_reg_embedding=0.0005, l2_reg_dnn=0.0005,
          pooling_width=( 2, 2,2),dnn_hidden_units=(4096,2048,1024,512,1,),dnn_dropout=0.4, task='binary')
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss="binary_crossentropy",metrics=['binary_crossentropy'])
earlysto                                                                                                                                                                                                                                                                                                                                                                      pping_callback = EarlyStopping(monitor='val_binary_crossentropy',verbose=1,patience=2,baseline=None)
history = model.fit(train_model_input, train[target].values,
                    batch_size=512, epochs=15,callbacks=[earlystopping_callback] ,verbose=2, validation_split=0.2)
pred_ans = model.predict(test_model_input, batch_size=256)

FGCNN start
origin_input shape: (None, 22, 40)
Train on 334458 samples, validate on 83615 samples
Epoch 1/15


/home/yxy/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/yxy/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


334458/334458 - 359s - loss: 1.2049 - binary_crossentropy: 0.3981 - val_loss: 0.4159 - val_binary_crossentropy: 0.3885
Epoch 2/15
334458/334458 - 346s - loss: 0.3911 - binary_crossentropy: 0.3797 - val_loss: 0.3935 - val_binary_crossentropy: 0.3860
Epoch 3/15
334458/334458 - 345s - loss: 0.3844 - binary_crossentropy: 0.3761 - val_loss: 0.3935 - val_binary_crossentropy: 0.3850
Epoch 4/15
334458/334458 - 345s - loss: 0.3839 - binary_crossentropy: 0.3746 - val_loss: 0.3946 - val_binary_crossentropy: 0.3853
Epoch 5/15
334458/334458 - 347s - loss: 0.3837 - binary_crossentropy: 0.3740 - val_loss: 0.3940 - val_binary_crossentropy: 0.3844
Epoch 6/15
334458/334458 - 347s - loss: 0.3836 - binary_crossentropy: 0.3735 - val_loss: 0.3945 - val_binary_crossentropy: 0.3846
Epoch 7/15
334458/334458 - 347s - loss: 0.3835 - binary_crossentropy: 0.3733 - val_loss: 0.3943 - val_binary_crossentropy: 0.3843
Epoch 8/15
334458/334458 - 348s - loss: 0.3835 - binary_crossentropy: 0.3732 - val_loss: 0.3944 - val

In [16]:
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

test LogLoss 0.3873
test AUC 0.7623


In [ ]:
model = FGCNN(linear_feature_columns, dnn_feature_columns,conv_kernel_width=(7, 7, 7), conv_filters=(14, 16, 18),
          new_maps=(3, 3, 3),
          pooling_width=( 2, 2,2),dnn_dropout=0.4, task='binary')
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss="binary_crossentropy",metrics=['binary_crossentropy'])
earlystopping_callback = EarlyStopping(monitor='val_binary_crossentropy',verbose=1,patience=1,baseline=None)
history = model.fit(train_model_input, train[target].values,
                    batch_size=512, epochs=15,callbacks=[earlystopping_callback], verbose=2, validation_split=0.2)
pred_ans = model.predict(test_model_input, batch_size=256)